## EXPERIMENTS

IMPORT DATA 


In [6]:
import pandas as pd


file_path = "/Users/USER/Desktop/ΓΙΑΝΝΗΣ/Msc BIG DATA ANALYTICS/THESIS/Crimes from 2020.csv"
df = pd.read_csv(file_path, delimiter=';')

In [7]:
df.head(10) 

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,3/1/2020 0:00,3/1/2020 0:00,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,340.375,-1.183.506
1,200106753,2/9/2020 0:00,2/8/2020 0:00,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,340.444,-1.182.628
2,200320258,11/11/2020 0:00,11/4/2020 0:00,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.021,-1.183.002
3,200907217,5/10/2023 0:00,3/10/2020 0:00,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,341.576,-1.184.387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,340.944,-1.183.277
5,231808869,4/4/2023 0:00,12/1/2020 0:00,2300,18,Southeast,1826,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,9900 COMPTON AV,NaN,339.467,-1.182.463
6,230110144,4/4/2023 0:00,7/3/2020 0:00,900,1,Central,182,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1100 S GRAND AV,NaN,340.415,-118.262
7,220314085,07/22/2022 12:00:00 AM,5/12/2020 0:00,1110,3,Southwest,303,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,2500 S SYCAMORE AV,NaN,340.335,-1.183.537
8,231309864,04/28/2023 12:00:00 AM,12/9/2020 0:00,1400,13,Newton,1375,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1300 E 57TH ST,NaN,339.911,-1.182.521
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,19,Mission,1974,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,9000 CEDROS AV,NaN,342.336,-1.184.535


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918443 entries, 0 to 918442
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           918443 non-null  int64  
 1   Date Rptd       918443 non-null  object 
 2   DATE OCC        918443 non-null  object 
 3   TIME OCC        918443 non-null  int64  
 4   AREA            918443 non-null  int64  
 5   AREA NAME       918443 non-null  object 
 6   Rpt Dist No     918443 non-null  int64  
 7   Part 1-2        918443 non-null  int64  
 8   Crm Cd          918443 non-null  int64  
 9   Crm Cd Desc     918443 non-null  object 
 10  Mocodes         790151 non-null  object 
 11  Vict Age        918443 non-null  int64  
 12  Vict Sex        796365 non-null  object 
 13  Vict Descent    796356 non-null  object 
 14  Premis Cd       918433 non-null  float64
 15  Premis Desc     917886 non-null  object 
 16  Weapon Used Cd  317500 non-null  float64
 17  Weapon Des

In [15]:
df_missingvalues = df.isnull().sum()
df_values = len(df)

print(df_missingvalues,df_values)


DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           128292
Vict Age               0
Vict Sex          122078
Vict Descent      122087
Premis Cd             10
Premis Desc          557
Weapon Used Cd    600943
Weapon Desc       600943
Status                 0
Status Desc            0
Crm Cd 1              11
Crm Cd 2          851659
Crm Cd 3          916192
Crm Cd 4          918378
LOCATION               0
Cross Street      774119
LAT                    0
LON                    0
dtype: int64 918443
